In [1]:
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from sklearn.manifold import TSNE
import itertools
import collections
from collections import Counter
from gensim import corpora, models
from gensim.models import Word2Vec, word2vec, LdaModel
import optuna
from hyperopt import hp, tpe, fmin
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime
# import spacy
from nltk import pos_tag
import pickle


In [3]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('drive/MyDrive/5.TopicModeling/Data/final.csv', encoding='latin-1')
df['tweet_created_at'] = pd.to_datetime(df["tweet_created_at"])
df['tweet_created_at'] = df['tweet_created_at'].dt.strftime('%Y-%m-%d')
df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
0,0,9.041318e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
1,1,9.041317e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
2,2,9.041309e+17,California on fire and Texas under water smh,2017-09-02,NaN,en,True,0.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,california fire texas water smh,"['california', 'texas', 'water', 'smh']","[('california', 'NN'), ('fire', 'NN'), ('texas...","['california', 'fire', 'texas', 'water', 'smh']","['california', 'fire', 'texas', 'water', 'smh']"
3,3,9.041304e+17,MVC: 6000 block of Tom Higgins Lane. Lake Shan...,2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,mvc 6000 block tom higgins lane lake shannon b...,"['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","[('mvc', 'NN'), ('6000', 'CD'), ('block', 'NN'...","['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","['block', 'tom', 'lane', 'lake', 'boat', 'laun..."
4,4,9.041302e+17,"Wildlife brings smoke, ash, unhealthful air qu...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildlife brings smoke ash unhealthful air qual...,"['wildlife', 'brings', 'smoke', 'ash', 'unheal...","[('wildlife', 'JJ'), ('brings', 'NNS'), ('smok...","['wildlife', 'brings', 'smoke', 'ash', 'unheal...","['wildlife', 'brings', 'smoke', 'ash', 'air', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301471,301471,9.415234e+17,"Pasadena, California â¢ \nJet Propulsion Labo...",2017-12-15,NaN,en,False,3.0,0.0,9.0,...,Mid-Wilshire,"Mid-Wilshire, Los Angeles",Verenigde Staten,US,neighborhood,pasadena california jet propulsion laboratory ...,"['pasadena', 'california', 'jet', 'propulsion'...","[('pasadena', 'NN'), ('california', 'NN'), ('j...","['pasadena', 'california', 'jet', 'propulsion'...","['california', 'jet', 'fire', 'personnel', 'as..."
301472,301472,9.419101e+17,So sad.........:(\nPlease help the family of t...,2017-12-16,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad please help family california firefighter ...,"['sad', 'help', 'family', 'california', 'firef...","[('sad', 'JJ'), ('please', 'NN'), ('help', 'VB...","['sad', 'please', 'help', 'family', 'californi...","['sad', 'please', 'help', 'family', 'californi..."
301473,301473,9.424684e+17,My heart goes out to all those affected by the...,2017-12-17,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,heart goes affected california wildfires sad,"['heart', 'goes', 'affected', 'california', 'w...","[('heart', 'NN'), ('goes', 'VBZ'), ('affected'...","['heart', 'go', 'affect', 'california', 'wildf...","['heart', 'go', 'affect', 'california', 'wildf..."
301474,301474,9.458634e+17,@realDonaldTrump Thank you Mr Trump for your i...,2017-12-27,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,thank mr trump incredible leadership divine sk...,"['thank', 'mr', 'trump', 'incredible', 'leader...","[('thank', 'JJ'), ('trump', 'NN'), ('incredibl...","['thank', 'trump', 'incredible', 'leadership',...","['thank', 'trump', 'incredible', 'leadership',..."


### separating data into three difference period

09/02/2017-10/07/2017: Before wildfire

10/08/2017-10/31/2017: During wildfire

11/01/2017-12/01/2017: After wildfire

In [6]:
#before wildfire
start_date = '2017-11-01'
end_date = '2017-12-01'
filtered_df = df[(df['tweet_created_at'] >= start_date) & (df['tweet_created_at'] <= end_date)]

# Word Distribution

In [7]:
wordcounter = Counter()
# Update the Counter with all tokens in the 'lemm_text' column
for tokens in filtered_df['lemm_text']:
    wordcounter.update(tokens)
filtered_df['lemm_textA'] = [[word for word in tokens if wordcounter[word] >= 100] for tokens in filtered_df['lemm_text']]
filtered_df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
76908,76908,9.258750e+17,ð¥FEATURED POST ð¥\n @firephoto25 - House...,2017-11-01,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,featured post house fire east lake rd conesus ...,"['featured', 'post', 'house', 'east', 'lake', ...","[('featured', 'VBN'), ('post', 'NN'), ('house'...","['feature', 'post', 'house', 'fire', 'east', '...","[[, ', f, e, a, t, u, r, e, ', ,, , ', p, o, ..."
76909,76909,9.258741e+17,"Just came from #fire in California, which kill...",2017-11-01,NaN,en,True,9.0,2.0,8.0,...,NaN,NaN,NaN,NaN,NaN,came fire california killed 40 amp burned 5000...,"['came', 'california', 'killed', 'amp', 'burne...","[('came', 'VBD'), ('california', 'NN'), ('kill...","['come', 'california', 'kill', 'amp', 'burned'...","[[, ', c, o, m, e, ', ,, , ', c, a, l, i, f, ..."
76910,76910,9.258737e+17,Last chance to win! $5/ticket. Proceeds to ben...,2017-11-01,NaN,en,True,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,last chance win 5ticket proceeds benefit ca wi...,"['chance', 'win', 'ticket', 'proceeds', 'benef...","[('last', 'JJ'), ('chance', 'NN'), ('win', 'NN...","['last', 'chance', 'win', 'ticket', 'proceeds'...","[[, ', l, a, s, t, ', ,, , ', c, h, a, n, c, ..."
76911,76911,9.258722e+17,Full containment reached on all North San Fran...,2017-11-01,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,full containment reached north san francisco b...,"['containment', 'reached', 'north', 'san', 'fr...","[('full', 'JJ'), ('containment', 'NN'), ('reac...","['full', 'containment', 'reach', 'north', 'san...","[[, ', f, u, l, l, ', ,, , ', c, o, n, t, a, ..."
76912,76912,9.258706e+17,California wildfire #Insurance #Claims top $3....,2017-11-01,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,california wildfire insurance claims top 33b,"['california', 'wildfire', 'insurance', 'claims']","[('california', 'NN'), ('wildfire', 'NN'), ('t...","['california', 'wildfire', 'top']","[[, ', c, a, l, i, f, o, r, n, i, a, ', ,, , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301231,301231,9.357814e+17,Health risks of toxic ash feared in wake of So...,2017-11-29,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,health risks toxic ash feared wake sonoma coun...,"['health', 'risks', 'toxic', 'ash', 'feared', ...","[('health', 'NN'), ('risks', 'NNS'), ('toxic',...","['health', 'risk', 'toxic', 'ash', 'fear', 'wa...","[[, ', h, e, a, l, t, h, ', ,, , ', r, i, s, ..."
301232,301232,9.357470e+17,I'll miss these guys (and a little sad we neve...,2017-11-29,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,ill miss guys little sad never got see steins ...,"['ill', 'miss', 'guys', 'little', 'sad', 'got'...","[('miss', 'NNS'), ('guys', 'NNS'), ('little', ...","['miss', 'guy', 'little', 'sad', 'never', 'get...","[[, ', m, i, s, s, ', ,, , ', g, u, y, ', ,, ..."
301233,301233,9.357392e+17,@Giants It's sad how a franchise can get ruine...,2017-11-29,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad franchise get ruined 2 seasons embarrass 2...,"['sad', 'franchise', 'ruined', 'seasons', 'emb...","[('sad', 'JJ'), ('franchise', 'NN'), ('get', '...","['sad', 'franchise', 'get', 'ruin', 'season', ...","[[, ', s, a, d, ', ,, , ', f, r, a, n, c, h, ..."
301234,301234,9.357290e+17,"â... but the fearful, and unbelieving, and t...",2017-11-29,NaN,en,True,0.0,0.0,2.0,...,Arlington,"Arlington, TX",Verenigde Staten,US,city,fearful unbelieving abominable murderers whore...,"['fearful', 'unbelieving', 'abominable', 'murd...","[('fearful', 'JJ'), ('unbelieving', 'VBG'), ('...","['fearful', 'unbelieving', 'abominable', 'murd...","[[, ', f, e, a, r, f, u, l, ', ,, , ', u, n, ..."


# Find dimension

In [8]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer(
                             max_features = 1000)

# document_texts = [' '.join(tokens) for tokens in df['lemm_text']]
document_texts = [tokens for tokens in filtered_df['lemm_text']]


# Fit the vectorizer on the 'lemm_text' column
vectorizer.fit(document_texts)

# Transform the 'lemm_text' column into a count vector representation
count_vector = vectorizer.transform(document_texts)

# Get the dimensions of the count vector
vector_dimensions = count_vector.shape

# Print the dimensions
print("Count Vector Dimensions:", vector_dimensions)


Count Vector Dimensions: (42970, 1000)


In [ ]:
# #try from 10 to 100 topics iterations
# X = count_vector

# topic_range = range(10,101,10)
# perplexities = []

# for i in topic_range:
#   ldask = LatentDirichletAllocation(n_components = i, random_state = 5566)
#   ldask.fit(X)
#   perplexity = ldask.perplexity(X)
#   perplexities.append(perplexity)
#   print("Number of topics: {}, Perplexity: {}".format(i, perplexity))


Number of topics: 10, Perplexity: 628.4994212484246
Number of topics: 20, Perplexity: 672.0784812420251
Number of topics: 30, Perplexity: 746.731150340783
Number of topics: 40, Perplexity: 762.3110080205156
Number of topics: 50, Perplexity: 800.7181092787931
Number of topics: 60, Perplexity: 836.4836847990097
Number of topics: 70, Perplexity: 846.417980406342
Number of topics: 80, Perplexity: 862.8064466548482
Number of topics: 90, Perplexity: 865.7336368598882
Number of topics: 100, Perplexity: 878.348750528098


In [ ]:
# # Print the top words for each topic

# ldask10 = LatentDirichletAllocation(n_components = 10, random_state = 5566)
# ldask10.fit(X)

# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(ldask10.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

In [ ]:
# # Define the objective function for optimization
# def objective(trial):
#     n_topics = trial.suggest_int('n_topics', 10, 100)
#     alpha = trial.suggest_uniform('alpha', 0.1, 1.0)
#     beta = trial.suggest_uniform('beta', 0.1, 1.0)

#     lda = LatentDirichletAllocation(
#         n_components=n_topics,
#         doc_topic_prior=alpha,
#         topic_word_prior=beta,
#         random_state=42
#     )

#     lda.fit(X)
#     perplexity = lda.perplexity(X)
#     return perplexity

# # Run the optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Print the best hyperparameters and objective value
# best_params = study.best_params
# best_value = study.best_value
# print("Best Hyperparameters: ", best_params)
# print("Best Objective Value: ", best_value)

In [ ]:
# # Print the top words for each topic

# ldask_mod = LatentDirichletAllocation(n_components = 32, random_state = 5566,
#                                       doc_topic_prior = 0.4109887406315537, topic_word_prior = 0.10680018587638036)
# ldask_mod.fit(X)

# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(ldask_mod.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

# Gensim LDA

In [9]:
# Create a list of lists from the preprocessed documents
# processed_docs = [doc.split() for doc in df['lemm_text']]
processed_docs = [re.sub(r'[\[\],]', '', doc).split() for doc in filtered_df['lemm_text']]
dictionary = corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)  # Limit the dictionary size
# Convert the dictionary into a bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# Initialize the LDA model with 10 top
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state = 5566)

In [10]:
# Increase the number of topics from 10 to 100
new_num_topics = 100

while num_topics < new_num_topics:
    # Increase the number of topics
    num_topics += 10

    # Re-train the LDA model with the updated number of topics
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

    # Evaluate the model's performance (using coherence score as an example)
    coherence_score = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v').get_coherence()

    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")

    # Repeat the process until the desired number of topics or satisfactory results are achieved

Number of topics: 20, Coherence Score: 0.33080766685934904
Number of topics: 30, Coherence Score: 0.3605315334977017
Number of topics: 40, Coherence Score: 0.37113320598517724
Number of topics: 50, Coherence Score: 0.40639165864588245
Number of topics: 60, Coherence Score: 0.4213070813374406
Number of topics: 70, Coherence Score: 0.4335149125261739
Number of topics: 80, Coherence Score: 0.4213631432527718
Number of topics: 90, Coherence Score: 0.3970970097650276
Number of topics: 100, Coherence Score: 0.3591229379650441


In [11]:
#try new topic model
lda_modelA = models.LdaModel(corpus, num_topics=70, id2word=dictionary)
for i,topic in lda_modelA.show_topics(formatted=True, num_topics=70, num_words=15):
    print(str(i)+": "+ topic)
    print()

0: 0.171*"'home'" + 0.124*"'fire'" + 0.106*"'alarm'" + 0.093*"'anything'" + 0.082*"'reason'" + 0.046*"'thank'" + 0.033*"'damage'" + 0.027*"'dad'" + 0.023*"'hang'" + 0.023*"'dumb'" + 0.023*"'ground'" + 0.022*"'coffee'" + 0.017*"'pls'" + 0.017*"'heat'" + 0.015*"'couple'"

1: 0.300*"'start'" + 0.177*"'catch'" + 0.134*"'break'" + 0.071*"'child'" + 0.067*"'fire'" + 0.036*"'illegal'" + 0.033*"'get'" + 0.029*"'immediately'" + 0.017*"'sing'" + 0.016*"'arrest'" + 0.011*"'muslim'" + 0.007*"'frank'" + 0.004*"'jose'" + 0.003*"'iran'" + 0.002*"'california'"

2: 0.309*"'take'" + 0.297*"'look'" + 0.108*"'mean'" + 0.085*"'fire'" + 0.024*"'choose'" + 0.020*"'fault'" + 0.015*"'rock'" + 0.015*"'tip'" + 0.011*"'bridge'" + 0.009*"'bye'" + 0.008*"'agency'" + 0.006*"'bang'" + 0.005*"'tom'" + 0.004*"'effective'" + 0.003*"'help'"

3: 0.347*"'set'" + 0.215*"'love'" + 0.122*"'fire'" + 0.045*"'red'" + 0.036*"'question'" + 0.027*"'history'" + 0.016*"'event'" + 0.016*"'warn'" + 0.014*"'count'" + 0.014*"'security'" 

#Parameter Modification (Optuna)

In [12]:
def objective(trial):
    num_topics = trial.suggest_int('num_topics', 10, 100)
    # iterations = trial.suggest_int('iterations', 50, 100)
    alpha = trial.suggest_categorical('alpha', ['symmetric', 'asymmetric'])
    eta = trial.suggest_float('eta', 0.1, 1.0)
    # rounded_eta = round(suggested_eta, 2)

    # Handle None values for hyperparameters
    # alpha = 1.0 if alpha is None else alpha
    # beta = 1.0 if beta is None else beta

    # Train the LDA model with the current hyperparameters
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                         alpha=alpha, eta=eta)

    # Calculate coherence score for the model
    coherence_model = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and coherence score
best_params = study.best_params
best_coherence = study.best_value
print("Best number of topics:", best_params['num_topics'])
print("Best alpha:", best_params['alpha'])
print("Best beta:", best_params['eta'])
print("Best coherence score:", best_coherence)


[I 2023-06-29 06:00:24,465] A new study created in memory with name: no-name-580ed84f-49ca-4fc1-b4bb-d131bae6fab2
[I 2023-06-29 06:00:55,080] Trial 0 finished with value: 0.4438618572199529 and parameters: {'num_topics': 39, 'alpha': 'asymmetric', 'eta': 0.6837418329323012}. Best is trial 0 with value: 0.4438618572199529.
[I 2023-06-29 06:01:31,265] Trial 1 finished with value: 0.37656214338340777 and parameters: {'num_topics': 90, 'alpha': 'asymmetric', 'eta': 0.4891981523122191}. Best is trial 0 with value: 0.4438618572199529.
[I 2023-06-29 06:01:48,634] Trial 2 finished with value: 0.35138731156995445 and parameters: {'num_topics': 17, 'alpha': 'asymmetric', 'eta': 0.23271756275000866}. Best is trial 0 with value: 0.4438618572199529.
[I 2023-06-29 06:02:23,647] Trial 3 finished with value: 0.36295320505871764 and parameters: {'num_topics': 77, 'alpha': 'symmetric', 'eta': 0.9818478942171744}. Best is trial 0 with value: 0.4438618572199529.
[I 2023-06-29 06:02:42,730] Trial 4 finishe

Best number of topics: 59
Best alpha: asymmetric
Best beta: 0.20735492548471762
Best coherence score: 0.5089624305217659


In [ ]:
# Best number of topics: 59
# Best alpha: asymmetric
# Best beta: 0.20735492548471762
# Best coherence score: 0.5089624305217659
modified_lda_model = LdaModel(corpus, id2word = dictionary,
                       num_topics = best_params['num_topics'], alpha = best_params['alpha'],
                       eta = best_params['eta'])
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=best_params['num_topics'], num_words=15):
    print(str(i)+": "+ topic)
    print()

In [14]:
# save model
with open('drive/MyDrive/5.TopicModeling/Model/lda_afterWildfire_mod', 'wb') as file:
    pickle.dump(modified_lda_model, file)

In [16]:
# Load the LDA model
model_path = 'drive/MyDrive/5.TopicModeling/Model/lda_afterWildfire_mod'
with open(model_path, 'rb') as file:
    modified_lda_model = pickle.load(file)

#implement model
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=59, num_words=15):
    print(str(i)+": "+ topic)
    print()

0: 0.083*"'fire'" + 0.028*"'west'" + 0.023*"'trump'" + 0.022*"'fan'" + 0.022*"'terrible'" + 0.021*"'role'" + 0.021*"'player'" + 0.018*"'something'" + 0.018*"'new'" + 0.016*"'playing'" + 0.015*"'cowboy'" + 0.015*"'say'" + 0.013*"'wait'" + 0.012*"'donald'" + 0.011*"'save'"

1: 0.072*"'fire'" + 0.026*"'someone'" + 0.023*"'as'" + 0.021*"'reaction'" + 0.019*"'friday'" + 0.018*"'show'" + 0.018*"'need'" + 0.018*"'amp'" + 0.015*"'do'" + 0.013*"'reason'" + 0.013*"'help'" + 0.013*"'kid'" + 0.012*"'today'" + 0.012*"'blow'" + 0.012*"'fuel'"

2: 0.079*"'fire'" + 0.029*"'smoke'" + 0.027*"'really'" + 0.025*"'work'" + 0.023*"'still'" + 0.023*"'shit'" + 0.021*"'fuck'" + 0.021*"'year'" + 0.019*"'get'" + 0.018*"'bad'" + 0.018*"'terrible'" + 0.015*"'people'" + 0.012*"'even'" + 0.012*"'need'" + 0.011*"'many'"

3: 0.378*"'smoke'" + 0.062*"'day'" + 0.032*"'cigarette'" + 0.020*"'gif'" + 0.014*"'hand'" + 0.009*"'lmao'" + 0.009*"'woman'" + 0.008*"'black'" + 0.008*"'hour'" + 0.008*"'decision'" + 0.007*"'second'"

# Topic Definition

### Topic 3: Cigarette Smoking

The most prominent keyword is "smoke," which is strongly associated with smoking. Additionally, words like "cigarette," "hand," and "smoking" further support this topic. Other keywords such as "day," "hour," and "insomnia" suggest a focus on the duration and potential negative consequences of smoking. However, it's important to note that without further context, the exact interpretation of the topic may vary.

### Topic 4: Negative Experience or Opinion Regarding a Fire Incident

The keywords "'terrible'," "'fire'," and "'hot'" indicate a negative sentiment associated with a fire. Other keywords such as "'bring'," "'money'," and "'room'" might suggest concerns related to financial or property damage caused by the fire. Additionally, terms like "'american'," "'government'," and "'idiot'" might indicate a criticism or negative perception of certain entities or individuals involved in the incident.

### Topic 12: Anxiety and Depression in the Workplace

The prominent keywords like "anxiety," "depression," "leave," and "job" indicate a discussion related to mental health issues, particularly anxiety and depression, in the context of a job or workplace. Other keywords like "tension," "forget," and "drop" further reinforce the theme of stress and mental strain. While there are a few unrelated keywords like "senator" and "firework," the overall pattern suggests a focus on mental health challenges in the workplace.

### Topic 15: Anger and Frustration

The prominent words such as "angry," "guy," and "stop" suggest a sense of anger or frustration. The presence of the word "fire" could potentially indicate an additional layer of intensity or danger associated with the topic. While other words like "hair," "sky," and "twin" are present, they do not seem to contribute significantly to the overall theme. Therefore, "Anger and Frustration" would be an appropriate label for this topic based on the given keywords.

### Topic 17: Movie Talk

The most prominent keyword is "'look'," which suggests a discussion or comment about a movie or a specific scene. Other keywords such as "'movie'," "'absolutely'," "'date'," and "'watched'" further support the topic of discussing or expressing opinions about movies. Although some keywords like "'bench'," "'promote'," and "'style'" are less specific to movies, they are outweighed by the movie-related keywords, indicating that the topic primarily revolves around movies.

### Topic 38: Worry and Emotional States

The most prominent keyword is "worry," which suggests that the topic revolves around concerns and anxieties. The presence of words like "miss," "believe," "bitch," and "hero" indicates emotional states and interpersonal dynamics. Other keywords such as "rain," "fire," and "dry" could potentially relate to environmental or metaphorical contexts.

### Topic 42: City Performance and Events

The keywords "city," "performance," and "door" suggest a focus on activities or events happening within a city. The presence of words like "Jones," "Monday," and "rapper" implies a connection to specific individuals or possibly a music-related event. Additional keywords like "failure," "responsibility," and "suffers" may indicate challenges or setbacks related to the mentioned performance or events. Considering these factors, the topic can be identified as "City Performance and Events."

### Topic 47: Car-related

Considering these keywords, it is possible that the topic could be related to a request or discussion involving a car and potentially a plea for assistance ("'pls'"). The mention of "'daughter'" might indicate a specific context or situation involving a person's daughter. The presence of other keywords like "'shooter'," "'cash'," and "'grenade'" is perplexing and doesn't provide much context to identify a specific topic.